# Classification by a neural network using Keras

## 0. Import packages and modules

In [ ]:
!pwd

In [ ]:
!hostname

In [ ]:
import os
# Limit log messages from Tensorflow
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import sklearn
import tensorflow
import keras

print(sklearn.__version__)
print(tensorflow.__version__)
print(keras.__version__)

In [ ]:
# print GPU info

print(tensorflow.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

## 1. Formulate/outline the problem: Penguin classification

## 2. Identify inputs and outputs

In [ ]:
# load penguins dataset

penguins = pd.read_csv('./penguins_dataset.csv')
penguins.head()

In [ ]:
# shape of dataset

penguins.shape

In [ ]:
# visualize dataset using pairplot

sns.pairplot(penguins[["species", "bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]], hue="species", height=2.5)

## 3. Prepare data

In [ ]:
# drop categorical columns

penguins_filtered = penguins.drop(columns=['island', 'sex'])
penguins_filtered.head(7)

In [ ]:
# drop the rows that have NaN values

penguins_filtered = penguins_filtered.dropna()
penguins_filtered.head(7)

In [ ]:
# extract columns corresponding to features

features = penguins_filtered.drop(columns=['species'])
features

In [ ]:
# encode categorical values to be target (label)

target = pd.get_dummies(penguins_filtered['species'])
print(target.head(5), '\n')   # print out the top 5 to see what it looks like
print(target.sample(7)) # randomly pickup 7 examples from the dataset

In [ ]:
# split dataset intro training and testing datasets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target,
            test_size=0.2, random_state=0, shuffle=True, stratify=target)

## 4. Build an architecture from scratch

In [ ]:
# set two random seeds, one for numpy and one for Keras/TensorFlow

from numpy.random import seed
seed(1)
keras.utils.set_random_seed(2)

In [ ]:
# construct three neural layers

inputs = keras.Input(shape=(X_train.shape[1],))
hidden_layer = keras.layers.Dense(10, activation="relu")(inputs)
output_layer = keras.layers.Dense(3, activation="softmax")(hidden_layer)

In [ ]:
# combine three neural layers into a neural network

model = keras.Model(inputs=inputs, outputs=output_layer)
model.summary()

## 5. Choose a loss function and optimizer

In [ ]:
# compile the model with loss function and optimizer

model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())

## 6. Train the model

In [ ]:
# train the model and save attributes in a history object

history = model.fit(X_train, y_train, epochs=100)

In [ ]:
# plot training loss to see how training progresses

sns.lineplot(x=history.epoch, y=history.history['loss'])

## 7. Perform a prediction/classification

In [ ]:
# make prediction on testing dataset

y_pred = model.predict(X_test)
prediction = pd.DataFrame(y_pred, columns=target.columns)
prediction

In [ ]:
# pickup the one with the largest probabilities

predicted_species = prediction.idxmax(axis="columns")
predicted_species

## 8. Measuring performance

In [ ]:
# create a confusion matrix

from sklearn.metrics import confusion_matrix

true_species = y_test.idxmax(axis="columns")
matrix = confusion_matrix(true_species, predicted_species)
print(matrix)

In [ ]:
# convert the confusion matrix to a pandas dataframe
confusion_df = pd.DataFrame(matrix, index=y_test.columns.values, columns=y_test.columns.values)

# set names of x and y axis, this helps with readability of heatmap.
confusion_df.index.name = 'True Label'
confusion_df.columns.name = 'Predicted Label'
confusion_df.head()

In [ ]:
# visualize the confusion matrix

sns.heatmap(confusion_df, annot=True, cmap='YlGn')

## 9. Refine the model

- Numbers of samples in training and testing datasets, we selected 80-20, choose 90-10.
- Numbers of hidden layers, and number of neurons in hidden layers.
    - Don't change number of neurons in input and output layers (why?)
- Choose another activation function.
- How about loss function and optimizer?
- Increase number of training epochs.

## 10. Share model

In [ ]:
# save the trained model

model.save('penguins_classification.keras')

In [ ]:
# load the saved model

pretrained_model = keras.models.load_model('penguins_classification.keras')

In [ ]:
# use the pretrained model here
y_pretrained_pred = pretrained_model.predict(X_test)
pretrained_prediction = pd.DataFrame(y_pretrained_pred, columns=target.columns.values)

# idxmax will select the column for each row with the highest value
pretrained_predicted_species = pretrained_prediction.idxmax(axis="columns")
print(pretrained_predicted_species)